# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
# Librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
#your code here

# Cargar el dataset
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")

# Limpieza de datos (rellenar NaNs, eliminar columnas irrelevantes)
spaceship.drop(['Name', 'Cabin', 'PassengerId'], axis=1, inplace=True)

# Imputación de valores faltantes
imputer = SimpleImputer(strategy='most_frequent')
spaceship[['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = imputer.fit_transform(spaceship[['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])

# Convertir columnas categóricas a dummies (One-Hot Encoding)
spaceship = pd.get_dummies(spaceship, columns=['HomePlanet', 'CryoSleep', 'Destination', 'VIP'], drop_first=True)

# Separar características (X) y objetivo (y)
X = spaceship.drop(['Transported'], axis=1)
y = spaceship['Transported'].astype(int)

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar características numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [4]:
#your code here

# Definir el modelo (Random Forest por ser el mejor hasta ahora)
rf = RandomForestClassifier(random_state=42)

# Ajustar el modelo con los datos de entrenamiento
rf.fit(X_train_scaled, y_train)

# Evaluar el modelo inicial en el conjunto de prueba
y_pred_rf_initial = rf.predict(X_test_scaled)

# Evaluar el modelo antes del ajuste de hiperparámetros
print("Resultados Random Forest antes de ajuste de hiperparámetros:\n", classification_report(y_test, y_pred_rf_initial))


Resultados Random Forest antes de ajuste de hiperparámetros:
               precision    recall  f1-score   support

           0       0.79      0.76      0.77       861
           1       0.77      0.80      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739



- Evaluate your model

In [5]:
#your code here

# Realizar predicciones con el modelo inicial
y_pred_rf_initial = rf.predict(X_test_scaled)

# Evaluar el modelo usando accuracy, precision, recall, F1-score
print("Resultados del modelo Random Forest antes del ajuste de hiperparámetros:")
print(classification_report(y_test, y_pred_rf_initial))

# Accuracy del modelo inicial
accuracy_rf_initial = accuracy_score(y_test, y_pred_rf_initial)
print(f"Accuracy del modelo inicial: {accuracy_rf_initial:.4f}")

# Mostrar matriz de confusión
print("Matriz de confusión del modelo inicial:")
print(confusion_matrix(y_test, y_pred_rf_initial))


Resultados del modelo Random Forest antes del ajuste de hiperparámetros:
              precision    recall  f1-score   support

           0       0.79      0.76      0.77       861
           1       0.77      0.80      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739

Accuracy del modelo inicial: 0.7798
Matriz de confusión del modelo inicial:
[[656 205]
 [178 700]]


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [6]:
#your code here

# Definir los hiperparámetros a ajustar para Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


- Run Grid Search

In [7]:
# Implementar GridSearchCV
grid_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Ajustar el modelo con el conjunto de entrenamiento
grid_rf.fit(X_train_scaled, y_train)

# Mostrar los mejores hiperparámetros
print(f"Mejores parámetros Random Forest: {grid_rf.best_params_}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Mejores parámetros Random Forest: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


- Evaluate your model

In [8]:
# Predicción y evaluación para Random Forest después del ajuste
y_pred_rf_tuned = grid_rf.best_estimator_.predict(X_test_scaled)
print("Resultados Random Forest después de ajuste de hiperparámetros:\n", classification_report(y_test, y_pred_rf_tuned))

Resultados Random Forest después de ajuste de hiperparámetros:
               precision    recall  f1-score   support

           0       0.81      0.75      0.78       861
           1       0.77      0.82      0.80       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739

